In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#import warnings
#warnings.filterwarnings('ignore',category=FutureWarning)

import numpy as np
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
import pathlib

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
import cv2
cap = cv2.VideoCapture(0)               #-- Webcam based-case
#cap = cv2.VideoCapture("<video-path>")  #-- video file based-case

#-- Control the camera resolution with CAM
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)

# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

print("[INFO] TF verion = ",tf.__version__)

[INFO] TF verion =  2.4.0-rc3


In [ ]:
#--- Model Preparation ----#
def load_model(model_name):
    
  #--- 2) tf 2.2 model base url
  #  base_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' #not working....!!!
  #  model_file = model_name + '.tar.gz'

#models\research\object_detection\local_models\trained_model_large_original_15000\saved_model
    model_dir = '../tensorflowModels/models/research/object_detection/local_models/'+model_name # efficientdet_d1
    model_dir = pathlib.Path(model_dir)/"saved_model"
    print('[INFO] Loading the modle from '+ str(model_dir))
    model = tf.saved_model.load(str(model_dir))

    return model

In [ ]:
 ## 확인용
    
    model_dir = '../tensorflowModels/models/research/object_detection/local_models/'
    model_dir = pathlib.Path(model_dir)/"saved_model"
    print('[INFO] Loading the modle from '+ str(model_dir))

[INFO] Loading the modle from ..\tensorflowModels\models\research\object_detection\local_models\saved_model


In [ ]:
# List of the strings that is used to add correct label for each box.
#PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
PATH_TO_LABELS = '../tensorflowModels/models/research/object_detection/local_models/snacks_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS)#, use_display_name=True)

# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
#PATH_TO_TEST_IMAGES_DIR = pathlib.Path('test_images')
#TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
#TEST_IMAGE_PATHS

#--- Detection --#
#\trained_model_large_original_15000
#model_name =  'faster_rcnn_resnet50_v1_640x640_coco17_tpu-8'#'centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8' #'efficientdet_d0_coco17_tpu-32' #'ssd_mobilenet_v1_coco_2017_11_17'
model_name =  'trained_model_large_original_15000' #'centernet_resnet50_v1_fpn_512x512_kpts_coco17_tpu-8' #'efficientdet_d0_coco17_tpu-32' #'ssd_mobilenet_v1_coco_2017_11_17'



print('[INFO] Downloading model and loading to network : '+ model_name)
detection_model = load_model(model_name)
#print(detection_model.signatures['serving_default'].inputs)
detection_model.signatures['serving_default'].output_dtypes
detection_model.signatures['serving_default'].output_shapes

[INFO] Downloading model and loading to network : trained_model_large_original_15000
[INFO] Loading the modle from ..\tensorflowModels\models\research\object_detection\local_models\trained_model_large_original_15000\saved_model


{'detection_anchor_indices': TensorShape([1, 100]),
 'detection_boxes': TensorShape([1, 100, 4]),
 'detection_classes': TensorShape([1, 100]),
 'detection_multiclass_scores': TensorShape([1, 100, 3]),
 'detection_scores': TensorShape([1, 100]),
 'num_detections': TensorShape([1]),
 'raw_detection_boxes': TensorShape([1, 49104, 4]),
 'raw_detection_scores': TensorShape([1, 49104, 3])}

In [ ]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
      # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
      # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

      # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

      # All outputs are batches tensors.
      # Convert to numpy arrays, and take index [0] to remove the batch dimension.
      # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy()
                     for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

      # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

      # Handle models with masks:
    if 'detection_masks' in output_dict:        
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict


In [ ]:

def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
  # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=8)

    display(Image.fromarray(image_np))

In [ ]:
def run_inference(model, cap):
    fn=0
    while cap.isOpened():
        ret, image_np = cap.read()
        # Actual detection.
        print("[INFO]"+str(fn)+"-th frame -- Running the inference and showing the result....!!!")
        output_dict = run_inference_for_single_image(model, image_np)
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
            image_np,
            output_dict['detection_boxes'],
            output_dict['detection_classes'],
            output_dict['detection_scores'],
            category_index,
            instance_masks=output_dict.get('detection_masks_reframed', None),
                    use_normalized_coordinates=True,
                    line_thickness=8)
        
        cv2.imshow('object_detection', cv2.resize(image_np, (1920,1080)))
            
        fn=fn+1
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cap.release()
            #cv2.destroyAllWindows()
            break

In [ ]:
##run!
run_inference(detection_model, cap)

[INFO]0-th frame -- Running the inference and showing the result....!!!
[INFO]1-th frame -- Running the inference and showing the result....!!!
[INFO]2-th frame -- Running the inference and showing the result....!!!
[INFO]3-th frame -- Running the inference and showing the result....!!!
[INFO]4-th frame -- Running the inference and showing the result....!!!
[INFO]5-th frame -- Running the inference and showing the result....!!!
[INFO]6-th frame -- Running the inference and showing the result....!!!
[INFO]7-th frame -- Running the inference and showing the result....!!!
[INFO]8-th frame -- Running the inference and showing the result....!!!
[INFO]9-th frame -- Running the inference and showing the result....!!!
[INFO]10-th frame -- Running the inference and showing the result....!!!
[INFO]11-th frame -- Running the inference and showing the result....!!!
[INFO]12-th frame -- Running the inference and showing the result....!!!
[INFO]13-th frame -- Running the inference and showing the re

[INFO]113-th frame -- Running the inference and showing the result....!!!
[INFO]114-th frame -- Running the inference and showing the result....!!!
[INFO]115-th frame -- Running the inference and showing the result....!!!
[INFO]116-th frame -- Running the inference and showing the result....!!!
[INFO]117-th frame -- Running the inference and showing the result....!!!
[INFO]118-th frame -- Running the inference and showing the result....!!!
[INFO]119-th frame -- Running the inference and showing the result....!!!
[INFO]120-th frame -- Running the inference and showing the result....!!!
[INFO]121-th frame -- Running the inference and showing the result....!!!
[INFO]122-th frame -- Running the inference and showing the result....!!!
[INFO]123-th frame -- Running the inference and showing the result....!!!
[INFO]124-th frame -- Running the inference and showing the result....!!!
[INFO]125-th frame -- Running the inference and showing the result....!!!
[INFO]126-th frame -- Running the infe

[INFO]224-th frame -- Running the inference and showing the result....!!!
[INFO]225-th frame -- Running the inference and showing the result....!!!
[INFO]226-th frame -- Running the inference and showing the result....!!!
[INFO]227-th frame -- Running the inference and showing the result....!!!
[INFO]228-th frame -- Running the inference and showing the result....!!!
[INFO]229-th frame -- Running the inference and showing the result....!!!
[INFO]230-th frame -- Running the inference and showing the result....!!!
[INFO]231-th frame -- Running the inference and showing the result....!!!
[INFO]232-th frame -- Running the inference and showing the result....!!!
[INFO]233-th frame -- Running the inference and showing the result....!!!
[INFO]234-th frame -- Running the inference and showing the result....!!!
[INFO]235-th frame -- Running the inference and showing the result....!!!
[INFO]236-th frame -- Running the inference and showing the result....!!!
[INFO]237-th frame -- Running the infe

[INFO]335-th frame -- Running the inference and showing the result....!!!
[INFO]336-th frame -- Running the inference and showing the result....!!!
[INFO]337-th frame -- Running the inference and showing the result....!!!
[INFO]338-th frame -- Running the inference and showing the result....!!!
[INFO]339-th frame -- Running the inference and showing the result....!!!
[INFO]340-th frame -- Running the inference and showing the result....!!!
[INFO]341-th frame -- Running the inference and showing the result....!!!
[INFO]342-th frame -- Running the inference and showing the result....!!!
[INFO]343-th frame -- Running the inference and showing the result....!!!
[INFO]344-th frame -- Running the inference and showing the result....!!!
[INFO]345-th frame -- Running the inference and showing the result....!!!
[INFO]346-th frame -- Running the inference and showing the result....!!!
[INFO]347-th frame -- Running the inference and showing the result....!!!
[INFO]348-th frame -- Running the infe

[INFO]446-th frame -- Running the inference and showing the result....!!!
[INFO]447-th frame -- Running the inference and showing the result....!!!
[INFO]448-th frame -- Running the inference and showing the result....!!!
[INFO]449-th frame -- Running the inference and showing the result....!!!
[INFO]450-th frame -- Running the inference and showing the result....!!!
[INFO]451-th frame -- Running the inference and showing the result....!!!
[INFO]452-th frame -- Running the inference and showing the result....!!!
[INFO]453-th frame -- Running the inference and showing the result....!!!
[INFO]454-th frame -- Running the inference and showing the result....!!!


In [ ]:
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        while True:
            ret,image_np=cap.read()
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
              [detection_boxes, detection_scores, detection_classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})
          # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
            cv2.imshow('object detection',cv2.resize(image_np,(1000,800)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

NameError: name 'detection_graph' is not defined